# On modelling optimization problems via Julia JuMP

## Prof. Mayron César O. Moreira 

**Universidade Federal de Lavras (UFLA)**  
**Department of Computer Science**  
**Lavras, Minas Gerais, Brazil**  

*Università degli Studi di Modena e Reggio Emilia (UNIMORE)*  
*Reggio Emilia, Italy*

## Assembly Line Worker Assignment and Balancing Problem (ALWABP)

Becker & Scholl (2006) indicate that SALBP characteristics are restrictive concerning assembly lines presented in real contexts. One of the main assumptions that we do not consider in SALBP is the fact that heterogeneity is presented while workers perform activities. Miralles et al. (2007) propose the first mathematical model that deal with an assembly line balancing composed entirely by people with disabilities. The challendge of this environment is that workers have a considerable variation of task executation times, and in some cases, there are workers that are not able to perform a set of tasks. As follows, we show an example of an instance for ALWABP (Moreira, 2015), with 3 workers, 3 stations and 9 tasks. 

In [67]:
const INF = 1000000 # Our infty

# Task executation time for each worker
tableTime = Dict(
    ("w1", "A") =>  5,
    ("w1", "B") =>  4,
    ("w1", "C") =>  11,
    ("w1", "D") =>  3,
    ("w1", "E") =>  4,
    ("w1", "F") =>  INF,
    ("w1", "G") =>  10,
    ("w1", "H") =>  30,
    ("w1", "I") =>  15,
    ("w2", "A") =>  3,
    ("w2", "B") =>  4,
    ("w2", "C") =>  10,
    ("w2", "D") =>  INF,
    ("w2", "E") =>  INF,
    ("w2", "F") =>  12,
    ("w2", "G") =>  3,
    ("w2", "H") =>  1,
    ("w2", "I") =>  9,
    ("w3", "A") =>  40,
    ("w3", "B") =>  INF,
    ("w3", "C") =>  2,
    ("w3", "D") =>  9,
    ("w3", "E") =>  7,
    ("w3", "F") =>  3,
    ("w3", "G") =>  25,
    ("w3", "H") =>  20,
    ("w3", "I") =>  35,
)

for (cnt, j) in enumerate(tableTime)
    println(cnt, " ", j)
end

1 ("w1", "C") => 11
2 ("w2", "B") => 4
3 ("w2", "C") => 10
4 ("w1", "A") => 5
5 ("w3", "B") => 1000000
6 ("w1", "I") => 15
7 ("w2", "E") => 1000000
8 ("w2", "G") => 3
9 ("w1", "G") => 10
10 ("w1", "B") => 4
11 ("w1", "E") => 4
12 ("w2", "I") => 9
13 ("w3", "A") => 40
14 ("w3", "H") => 20
15 ("w3", "I") => 35
16 ("w1", "H") => 30
17 ("w3", "C") => 2
18 ("w2", "D") => 1000000
19 ("w1", "D") => 3
20 ("w2", "F") => 12
21 ("w3", "E") => 7
22 ("w2", "A") => 3
23 ("w3", "D") => 9
24 ("w3", "F") => 3
25 ("w3", "G") => 25
26 ("w2", "H") => 1
27 ("w1", "F") => 1000000


![(Moreira, 2015)](images/alwabp.png)

### ALWABP-2

We present a three-index variable formulation proposed by Miralles et al. (2007) for ALWABP-2, the most common extension in this context. 

* **Importing libraries**

In [68]:
using JuMP
using Cbc
include("codes/instanceALs.jl")
;

* **Reading instance**

In [69]:
# ALWABP instance
fileInstance="instances/alwabp/heskia1"

io=open(fileInstance)

# Reading ALWABP instance
instance = readALWABP(io)
;

* **Creating model**

In [70]:
model = Model(with_optimizer(Cbc.Optimizer))
;

* **Sets and constants (inputs)**

    * **Additional parameters**
        * $W$: set oh heterogeneous workers ($|W|=|S|$);  
        * $p_{iw} \in \mathbb{R}_+$: task execution time to perform task $i$ by woker $w$ ($p_{wi} = \infty$ if worker $w$ does not perform task $i$);    

In [71]:
n = instance.n # number of tasks
w = instance.w # number of workers
m = instance.w # number of stations
P = [Vector{Int}() for _ in 1:n+1] # set of immediate predecessors
F = [Vector{Int}() for _ in 1:n+1] # set of immediate followers
c = instance.c #cycle time
p = instance.p

for i=1:n
    P[i] = copy(instance.P[i])
    F[i] = copy(instance.F[i])
end

S = collect(1:w) # set of stations
W = collect(1:w) # set of workers
N = collect(1:n) # set of tasks
;

* **Decision variables**

    * $x_{swi} \in \{0,1\}$: equal to one iff task $i$ is assigned to worker $w$ in station $s$;  
    * $y_{sw} \in \{0,1\}$: equal to one iff worker $w$ is assigned to station $s$;
    * $c \in \mathbb{R}_+$: cycle time of the line.

In [72]:
@variable(model, x[S,W,N], Bin)
@variable(model, y[S,W], Bin)
@variable(model, c >= 0)
;

* **Objective function**: minimize the cycle time

\begin{equation}
\min\,c
\end{equation}

In [73]:
@objective(model, Min, c)
;

* **Constraint:** all tasks must be assigned

\begin{equation}
\sum_{s \in S}\sum_{w \in W} x_{swi} = 1 \qquad \forall i \in N
\end{equation}

In [74]:
@constraint(model, [i in N], sum(x[s,w,i] for s in S, w in W) == 1)
;

* **Constraint:** a worker must be assigned to a station

\begin{equation}
\sum_{s \in S} y_{sw} = 1 \qquad \forall w \in W
\end{equation}

In [75]:
@constraint(model, [w in W], sum(y[s,w] for s in S) == 1)
;

* **Constraint:** a station must have a single worker

\begin{equation}
\sum_{w \in W} y_{sw} = 1 \qquad \forall s \in S
\end{equation}

In [76]:
@constraint(model, [s in S], sum(y[s,w] for w in W) == 1)
;

* **Constraint:** precedence relations are respected

\begin{equation}
\sum_{s \in S}\sum_{w \in W} sx_{swi} \le \sum_{s \in S}\sum_{w \in W} sx_{swj} \qquad \forall j \in N, \forall i \in D_j
\end{equation}

In [77]:
@constraint(model, [j in N, i in P[j]], sum(s*x[s,w,i] for s in S, w in W) <= sum(s*x[s,w,j] for s in S, w in W))
;

* **Constraint:** definition of cycle time as maximum load of a station

\begin{equation}
\sum_{i \in N} p_{iw}x_{swi} \le c \qquad \forall s \in S, \forall w \in W
\end{equation}

In [78]:
@constraint(model, [s in S, w in W], sum(p[i,w]*x[s,w,i] for i in N) <= c)
;

* **Constraint:** coupling constraints between $x$ and $y$

\begin{equation}
\sum_{i \in N} x_{swi} \le ny_{sw} \qquad \forall s \in S, \forall w \in W
\end{equation}

In [79]:
@constraint(model, [s in S, w in W], sum(x[s,w,i] for i in N) <= n*y[s,w])
;

* **Running the model**

In [80]:
optimize!(model)
;

Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 20.8883 - 0.01 seconds
Cgl0004I processed model has 107 rows, 465 columns (464 integer (464 of which binary)) and 2656 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 65 integers unsatisfied sum - 12.9479
Cbc0038I Pass   1: suminf.    5.64286 (24) obj. 5001 iterations 121
Cbc0038I Pass   2: suminf.    5.64286 (24) obj. 5001 iterations 5
Cbc0038I Pass   3: suminf.    4.48810 (18) obj. 168 iterations 41
Cbc0038I Pass   4: suminf.    3.04762 (17) obj. 168 iterations 19
Cbc0038I Pass   5: suminf.    3.04762 (17) obj. 168 iterations 2
Cbc0038I Pass   6: suminf.    3.02381 (15) obj. 168 iterations 23
Cbc0038I Pass   7: suminf.    2.76190 (16) obj. 168 iterations 18
Cbc0038I Pass   8: suminf.    2.52381 (16) obj. 168 iterations 30
Cbc0038I Pass   9: suminf.    2.35714 (16) obj. 168 iterations 

* **Printing variables**

In [81]:
println("Printing solution")
cOpt = JuMP.objective_value(model) # Optimal solution
println("Cycle time = ", cOpt)
tmp = 0 # Overload of each station
for s in S
    print("s", Int(s), "(w")
    for w in W
        if(value(y[s,w]) >= 0.9)
            print(w, "): ")
            for i=1:n
                if(value(x[s,w,i]) >= 0.9 && i != n+1)
                    print(i, " ")
                    tmp += p[i,w]
                end
            end
            break
        end
    end
    println("(", tmp, ")")
    tmp = 0
end

Printing solution
Cycle time = 94.0
s1(w3): 1 4 5 8 19 20 22 23 24 (91)
s2(w4): 2 6 9 10 12 13 (94)
s3(w2): 3 7 11 14 16 17 18 25 26 27 (93)
s4(w1): 15 21 28 (94)


## References

Becker, C., & Scholl, A. (2006). A survey on problems and methods in generalized assembly line balancing. European journal of operational research, 168(3), 694-715.

Miralles, C., Garcia-Sabater, J. P., Andres, C., & Cardos, M. (2007). Advantages of assembly lines in sheltered work centres for disabled. A case study. International Journal of Production Economics, 110(1-2), 187-197.

Moreira, M. C. O. (2015). Problema de balanceamento de linhas de produção e integração de trabalhadores. Tese (Doutorado em Ciências de Computação e Matemática Computacional) - Instituto de Ciências Matemáticas e de Computação, Universidade de São Paulo, São Carlos. Disponível em: http://www.teses.usp.br/teses/disponiveis/55/55134/tde-08012016-145627/pt-br.php